In [ ]:
import re
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words_eng = stopwords.words("english")
stop_words_ger = stopwords.words("german")
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

url_eng = r'/content/drive/MyDrive/TextSent/training set.csv'
url_eng_test = r'/content/drive/MyDrive/TextSent/testing set.csv'
url_ger = r'/content/drive/MyDrive/TextSent/RP-Mod-Crowd.csv'


df_eng = pd.read_csv(url_eng)
df_eng_test = pd.read_csv(url_eng_test)
df_ger = pd.read_csv(url_ger, index_col=0)

ValueError: mount failed

In [ ]:
df_eng = df_eng._append(df_eng_test,ignore_index=True)
df_eng.dropna()

df_eng['racist'] = df_eng['categories'] #add new column
df_eng['sexist'] = df_eng['categories'] #add new column

df_eng = df_eng.rename(columns={ 'tweet':'text'})
df_eng['racist'] = df_eng['racist'].replace('Race', 1).apply(lambda x: 0 if x != 1 else x)
df_eng['sexist'] = df_eng['sexist'].replace({'Gender'}, 1).apply(lambda x: 0 if x != 1 else x)

In [ ]:
df_ger.dropna()
df_ger = df_ger[['Text','Reject Newspaper', 'Sexism Count Crowd', 'Racism Count Crowd']]
df_ger = df_ger.rename(columns={'Sexism Count Crowd':'sexist','Racism Count Crowd':'racist','Text':'text'})
df_ger['racist'] = df_ger['racist'].apply(lambda x: 1 if x > 1 else 0)
df_ger['sexist'] = df_ger['sexist'].apply(lambda x: 1 if x > 1 else 0)

In [ ]:
df_ger.text=df_ger.text.astype(str)
df_ger = df_ger[df_ger['text'].map(len) < 200]
df_ger = df_ger[df_ger['text'].map(len) > 30]

In [ ]:
pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 766.4 kB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

df = df_ger

list_of_languages = ["it", "es", "fr", "en", "pt"]
df_augmented_full = df.copy()

def data_augmentation(text, translation_language):

  text = GoogleTranslator(source="de", target=translation_language).translate(text)
  de_text = GoogleTranslator(source=translation_language, target="de").translate(text)
  return de_text

In [ ]:
for language in list_of_languages:

      augmented_list_sexist = df[df.sexist == True].text.apply(lambda x: data_augmentation(x, language))

      cols = ["text", "sexist"]
      df_augmented_language = pd.DataFrame(augmented_list_sexist, columns=cols)
      df_augmented_language["sexist"] = 1

      df_augmented_full = pd.concat([df_augmented_full, df_augmented_language], axis=0)

In [ ]:
df_r = df[df.racist == True]
for language in list_of_languages:

      augmented_list_racist = df_r.text.apply(lambda x: data_augmentation(x, language))

      cols = ["text", "racist"]
      df_augmented_language_racist = pd.DataFrame(augmented_list_racist, columns=cols)
      df_augmented_language_racist["racist"] = 1

      df_augmented_full = pd.concat([df_augmented_full, df_augmented_language_racist], axis=0)

In [ ]:
df_eng_pp = preprocess_dataframe_eng(df_eng)
df_eng_pp.text=df_eng.text.astype(str)

In [ ]:
# Export
df_augmented_full.to_csv('/content/drive/MyDrive/TextSent/augmented_german_data.csv', index=False)
df_eng_pp.to_csv('/content/drive/MyDrive/TextSent/df_eng_pp.csv', index=False)